# Imports

In [1]:
import cv2
import numpy as np
import time          # for runtime measuring

# Part 1

In [2]:
def sum_matrix_conv(matrix1, matrix2):
    # Number of rows in the first matrix
    dim = len(matrix1)
    result = 0

    # Perform matrix convolution
    for i in range(dim):
        for j in range(dim):
            result += matrix1[i][j] * matrix2[i][j]

    return result

In [3]:
# pad variable is an int indicating the padding kind to add - 0: zero padding, 1:replicate padding, 2: mirror padding
def my_imfilter(s, fil, pad):
    in_dim = s.shape[0]
    fil_dim = len(fil[0])
    pad_num = int(fil_dim/2)
    
    # creating an array that will be used as the padded image
    pad_im = np.zeros((in_dim + 2 * pad_num, in_dim + 2 * pad_num), dtype = np.uint8)
    
    if (pad == 0): # zero padding
        pad_im[pad_num : in_dim + pad_num, pad_num : in_dim + pad_num] = s
        
    if (pad == 1): # replicate padding
        pad_im[pad_num : in_dim + pad_num, pad_num : in_dim + pad_num] = s
        # corners
        pad_im[ : pad_num,  : pad_num] = s[0][0]
        pad_im[ : pad_num, in_dim + pad_num : ] = s[0][in_dim-1]
        pad_im[in_dim + pad_num : , : pad_num] = s[in_dim-1][0]
        pad_im[in_dim + pad_num : , in_dim + pad_num : ] = s[in_dim-1][in_dim-1]
        # columns
        pad_im[pad_num : in_dim + pad_num, : pad_num] = s[:, 0][:, np.newaxis]
        pad_im[pad_num : in_dim + pad_num, in_dim + pad_num : ] = s[:, -1][:, np.newaxis]
        # rows
        pad_im[ : pad_num, pad_num : in_dim + pad_num] = s[0, :]
        pad_im[in_dim + pad_num : , pad_num : in_dim + pad_num] = s[-1, :]
        
    if (pad == 2): # mirror padding
        pad_im[pad_num : in_dim + pad_num, pad_num : in_dim + pad_num] = s
        # sides
        pad_im[pad_num : in_dim + pad_num, : pad_num] = s[:, pad_num:0 :-1]
        pad_im[pad_num : in_dim + pad_num, in_dim + pad_num : ] = s[:, in_dim - 2 : in_dim - pad_num - 2 :-1]
        # top & bottom
        pad_im[ : pad_num , : ] = pad_im[2 * pad_num : pad_num :-1, : ]
        pad_im[pad_num + in_dim :  , : ] = pad_im[pad_num + in_dim - 2 : in_dim-2 :-1, : ]

    out_image = np.zeros((in_dim, in_dim), dtype = np.uint8)
    
    for i in range (in_dim):
        for j in range (in_dim):
            out_image[i][j] = sum_matrix_conv(pad_im[i : i + fil_dim, j : j + fil_dim], fil)
            if (out_image[i][j] > 256):
                out_image[i][j] = 256
            if (out_image[i][j] < 0):
                out_image[i][j] = 0
    
    return out_image

# Part 2

In [4]:
# nlf variable is a string that indicates what non-linear filter will be applied.
# we implemented median filter and average filter.
def my_im_nl_filter(s, m, n, nlf, pad):
    in_dim = s.shape[0]
    pad_rows = int(m/2)
    pad_cols = int(n/2)
    
    # creating an array that will be used as the padded image
    pad_im = np.zeros((in_dim + 2 * pad_rows, in_dim + 2 * pad_cols), dtype = np.uint8)
    
    if (pad == 0): # zero padding
        pad_im[pad_rows : in_dim + pad_rows, pad_cols : in_dim + pad_cols] = s
        
    if (pad == 1): # replicate padding
        pad_im[pad_rows : in_dim + pad_rows, pad_cols : in_dim + pad_cols] = s
        # corners
        pad_im[ : pad_rows,  : pad_cols] = s[0][0]
        pad_im[ : pad_rows, in_dim + pad_cols : ] = s[0][in_dim-1]
        pad_im[in_dim + pad_rows : , : pad_cols] = s[in_dim-1][0]
        pad_im[in_dim + pad_rows : , in_dim + pad_cols : ] = s[in_dim-1][in_dim-1]
        # columns
        pad_im[pad_rows : in_dim + pad_rows, : pad_cols] = s[:, 0][:, np.newaxis]
        pad_im[pad_rows : in_dim + pad_rows, in_dim + pad_cols : ] = s[:, -1][:, np.newaxis]
        # rows
        pad_im[ : pad_rows, pad_cols : in_dim + pad_cols] = s[0, :]
        pad_im[in_dim + pad_rows : , pad_cols : in_dim + pad_cols] = s[-1, :]
        
    if (pad == 2): # mirror padding
        pad_im[pad_rows : in_dim + pad_rows, pad_cols : in_dim + pad_cols] = s
        # sides
        pad_im[pad_rows : in_dim + pad_rows, : pad_cols] = s[:, pad_cols:0 :-1]
        pad_im[pad_rows : in_dim + pad_rows, in_dim + pad_cols : ] = s[:, in_dim - 2 : in_dim - pad_cols - 2 :-1]
        # top & bottom
        pad_im[ : pad_rows , : ] = pad_im[2 * pad_rows : pad_rows :-1, : ]
        pad_im[pad_rows + in_dim :  , : ] = pad_im[pad_rows + in_dim - 2 : in_dim-2 :-1, : ]

    out_image = np.zeros((in_dim, in_dim), dtype = np.uint8)
    
    if (nlf == 'median'):
        for i in range (in_dim):
            for j in range (in_dim):
                out_image[i][j] = np.median(pad_im[i : i + m, j : j + n])
    
    if (nlf == 'average'):
        for i in range (in_dim):
            for j in range (in_dim):
                out_image[i][j] = int(np.average(pad_im[i : i + m, j : j + n]))
    
    return out_image

In [5]:
def add_salt_and_pepper_noise(image, salt_prob, pepper_prob):    
    row, col = image.shape
    s_vs_p = 0.5
    
    # Salt noise
    num_salt = np.ceil(salt_prob * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    image[coords[0], coords[1]] = 255

    # Pepper noise
    num_pepper = np.ceil(pepper_prob * image.size * (1.0 - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    image[coords[0], coords[1]] = 0

    return image

# "Main"

In [6]:
img=cv2.imread("lemons.jpg")
img = cv2.resize(img, (256, 256))
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

In [7]:
sharpen = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]]) # One Pass Laplace filter 3x3
smoothen = np.array([[1/9, 1/9, 1/9], [1/9, 1/9, 1/9], [1/9, 1/9, 1/9]]) # Smooth filter 3x3

# getting the time before and after each filter so we can measure the runtime of our functions
time1 = time.time()
sharped_image = my_imfilter(gray_img, sharpen, 0)
time2 = time.time()
smoothed_image = my_imfilter(gray_img, smoothen, 0)
time3 = time.time()
lib_smooth = cv2.filter2D(gray_img, -1, smoothen)
time4 = time.time()
lib_sharp = cv2.filter2D(gray_img, -1, sharpen)
time5 = time.time()

print("Our sharpening time is: {} seconds, the library filter2D sharpening time is: {} seconds"
      .format(time2-time1, time5-time4))
print("Our smoothing time is: {} seconds, the library filter2D smoothing time is: {} seconds"
      .format(time3-time2, time4-time3))

# showing and saving the results of each filter we applied
cv2.imshow("Grayscale", gray_img)
cv2.imwrite('gray_img.jpg', gray_img)

cv2.imshow("Sharpened", sharped_image)
cv2.imwrite('sharped_image.jpg', sharped_image)

cv2.imshow("Smoothed", smoothed_image)
cv2.imwrite('smoothed_image.jpg', smoothed_image)

cv2.imshow("Lib Smoothen", lib_smooth)
cv2.imwrite('lib_smooth.jpg', lib_smooth)

cv2.imshow("Lib Sharpen", lib_sharp)
cv2.imwrite('lib_sharp.jpg', lib_sharp)

cv2.waitKey(0)
cv2.destroyAllWindows()

Our sharpening time is: 1.3822879791259766 seconds, the library filter2D sharpening time is: 0.0 seconds
Our smoothing time is: 1.422196865081787 seconds, the library filter2D smoothing time is: 0.0009982585906982422 seconds


In [9]:
# Add salt and pepper noise
noisy_image = add_salt_and_pepper_noise(gray_img, salt_prob=0.1, pepper_prob=0.1)

# Save or display the result
cv2.imwrite('noisy_image.jpg', noisy_image)
cv2.imshow('Noisy Image', noisy_image)

smoothed_snp = my_imfilter(noisy_image, smoothen, 0)

cv2.imwrite('smoothed_snp.jpg', smoothed_snp)
cv2.imshow('Smoothed SnP', smoothed_snp)

median_im = my_im_nl_filter(gray_img, 3, 5, 'median', 0)

cv2.imwrite('median_im.jpg', median_im)
cv2.imshow('Median_im', median_im)

cv2.waitKey(0)
cv2.destroyAllWindows()